# Enrolment Data Cleaning Project
This notebook covers the step-by-step process of cleaning and merging three enrolment datasets.

In [ ]:
# Step 1: Import required libraries
import pandas as pd
import numpy as np
import os

print("Libraries imported successfully.")

## Step 2: Define Cleaning Rules

In [ ]:
# State Mapping Dictionary
state_corrections = {
    '100000': None,  # Will be dropped
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman and Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'The Dadra And Nagar Haveli And Daman And Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Jammu & Kashmir': 'Jammu and Kashmir',
    'Jammu And Kashmir': 'Jammu and Kashmir',
    'ODISHA': 'Odisha',
    'Orissa': 'Odisha',
    'Pondicherry': 'Puducherry',
    'WEST BENGAL': 'West Bengal',
    'WESTBENGAL': 'West Bengal',
    'West  Bengal': 'West Bengal',
    'West Bangal': 'West Bengal',
    'West bengal': 'West Bengal',
    'Westbengal': 'West Bengal',
    'andhra pradesh': 'Andhra Pradesh'
}

# District Mapping Dictionary
district_corrections = {
    # Remove special chars and standardize
    'Bagalkot *': 'Bagalkot',
    'Chamarajanagar *': 'Chamarajanagar',
    'Chamrajnagar': 'Chamarajanagar',
    'Chickmagalur': 'Chikkamagaluru',
    'Chikmagalur': 'Chikkamagaluru',
    'Chikkamagaluru': 'Chikkamagaluru',
    'Davangere': 'Davanagere',
    'Hasan': 'Hassan',
    'Haveri *': 'Haveri',
    'Ramanagar': 'Ramanagara',
    'Tumkur': 'Tumakuru',
    'Bagpat': 'Baghpat',
    'Bulandshahar': 'Bulandshahr',
    'Kushinagar *': 'Kushinagar',
    'Kushi Nagar': 'Kushinagar',
    'Mahrajganj': 'Maharajganj',
    'Shrawasti': 'Shravasti',
    'Siddharth Nagar': 'Siddharthnagar',
    'Aurangabad(BH)': 'Aurangabad',
    'Aurangabad(bh)': 'Aurangabad',
    'Purba Champaran': 'East Champaran',
    'Purbi Champaran': 'East Champaran',
    'Samstipur': 'Samastipur',
    'Sheikpura': 'Sheikhpura',
    'Ahmed Nagar': 'Ahmednagar',
    'Ahmadnagar': 'Ahmednagar',
    'Buldhana': 'Buldana',
    'Chatrapati Sambhaji Nagar': 'Chhatrapati Sambhajinagar',
    'Gondiya': 'Gondia',
    'Gondiya *': 'Gondia',
    'Hingoli *': 'Hingoli',
    'Nandurbar *': 'Nandurbar',
    'Washim *': 'Washim',
    'Jhajjar *': 'Jhajjar',
    'Yamuna Nagar': 'Yamunanagar',
    'Chittaurgarh': 'Chittorgarh',
    'Deeg\xa0': 'Deeg',
    'Jalore': 'Jalor',
    'Jhunjhunun': 'Jhunjhunu',
    'S.A.S Nagar(Mohali)': 'SAS Nagar (Mohali)',
    'Ashok Nagar': 'Ashoknagar',
    '24 Paraganas North': 'North 24 Parganas',
    'North Twenty Four Parganas': 'North 24 Parganas',
    'Barddhaman': 'Bardhaman',
    'Cooch Behar': 'Coochbehar',
    'East Midnapur': 'East Midnapore',
    'Hooghiy': 'Hooghly',
    'Maldah': 'Malda',
    'Puruliya': 'Purulia',
    'South 24 Pargana': 'South 24 Parganas',
    'South Twenty Four Parganas': 'South 24 Parganas',
    'hooghly': 'Hooghly',
    'Hardwar': 'Haridwar',
    'Ahmadabad': 'Ahmedabad',
    'Banas Kantha': 'Banaskantha',
    'Panch Mahals': 'Panchmahals',
    'Sabar Kantha': 'Sabarkantha',
    'Surendra Nagar': 'Surendranagar',
    'Ananthapur': 'Anantapur',
    'Ananthapuramu': 'Anantapur',
    'chittoor': 'Chittoor',
    'K.V.Rangareddy': 'Rangareddy',
    'K.v. Rangareddy': 'Rangareddy',
    'Karim Nagar': 'Karimnagar',
    'Mahabub Nagar': 'Mahabubnagar',
    'Mahbubnagar': 'Mahabubnagar',
    'rangareddi': 'Rangareddy',
    'Visakhapatanam': 'Visakhapatnam',
    'Kancheepuram': 'Kanchipuram',
    'Kanniyakumari': 'Kanyakumari',
    'Thiruvallur': 'Tiruvallur',
    'Tirupathur': 'Tirupattur',
    'Tiruvarur': 'Thiruvarur',
    'Villupuram': 'Viluppuram',
    'Janjgir - Champa': 'Janjgir Champa',
    'Janjgir-champa': 'Janjgir Champa',
    'Mohalla-Manpur-Ambagarh Chowki': 'Mohla-Manpur-Ambagarh Chowki',
    'Bokaro *': 'Bokaro',
    'East Singhbum': 'East Singhbhum',
    'Garhwa *': 'Garhwa',
    'Hazaribag': 'Hazaribagh',
    'Koderma': 'Kodarma',
    'Pakaur': 'Pakur',
    'Palamau': 'Palamu',
    'Sahebganj': 'Sahibganj',
    'Seraikela-Kharsawan': 'Seraikela Kharsawan',
    'Jangoan': 'Jangaon',
    'Baramula': 'Baramulla',
    'Bandipore': 'Bandipora',
    'Badgam': 'Budgam',
    'Gurgaon': 'Gurugram',
    'Gulbarga': 'Kalaburagi',
    'Mysore': 'Mysuru',
    'Belgaum': 'Belagavi',
    'Bellary': 'Ballari',
    'Bijapur': 'Vijayapura',
    'Shimoga': 'Shivamogga',

    # Chhattisgarh Corrections (Mapping to user's 33 districts)
    'Balod': 'Balod',
    'Baloda Bazar': 'Baloda Bazar-Bhatapara',
    'Baloda Bazar-Bhatapara': 'Baloda Bazar-Bhatapara',
    'Balrampur': 'Balrampur',
    'Bastar': 'Bastar',
    'Bemetara': 'Bemetara',
    'Bijapur': 'Bijapur',
    'Bilaspur': 'Bilaspur',
    'Dantewada': 'Dantewada',
    'Dakshin Bastar Dantewada': 'Dantewada',
    'Dhamtari': 'Dhamtari',
    'Durg': 'Durg',
    'Gariaband': 'Gariaband',
    'Gaurela-Pendra-Marwahi': 'Gaurela-Pendra-Marwahi',
    'Gaurella Pendra Marwahi': 'Gaurela-Pendra-Marwahi',
    'Janjgir-Champa': 'Janjgir-Champa',
    'Janjgir Champa': 'Janjgir-Champa',
    'Janjgir - Champa': 'Janjgir-Champa',
    'Jashpur': 'Jashpur',
    'Kabirdham': 'Kabirdham (Kawardha)',
    'Kabeerdham': 'Kabirdham (Kawardha)',
    'Kabirdham': 'Kabirdham (Kawardha)',
    'Kanker': 'Kanker',
    'Uttar Bastar Kanker': 'Kanker',
    'Kondagaon': 'Kondagaon',
    'Korba': 'Korba',
    'Khairagarh-Chhuikhadan-Gandai': 'Khairagarh-Chhuikhadan-Gandai',
    'Rajnandgaon': 'Rajnandgaon',
    'Sarangarh-Bilaigarh': 'Sarangarh-Bilaigarh',
    'Sakti': 'Sakti',
    'Sukma': 'Sukma',
    'Surajpur': 'Surajpur',
    'Surguja': 'Surguja',
    'Ambikapur': 'Surguja', # Ambikapur is HQ of Surguja, data often mixes these
    'Raipur': 'Raipur',
    'Raigarh': 'Raigarh',
    'Mahasamund': 'Mahasamund',
    'Korea': 'Koriya (Korea)',
    'Koriya': 'Koriya (Korea)',
    'Manendragarh-Chirmiri-Bharatpur': 'Manendragarh-Chirmiri-Bharatpur',
    'Mohla-Manpur-Ambagarh Chowki': 'Mohla-Manpur-Ambagarh Chowki',
    'Mohla Manpur Ambagarh Chowki': 'Mohla-Manpur-Ambagarh Chowki',
    'Narayanpur': 'Narayanpur'
}

## Step 3: Process Enrolment Data 1

In [ ]:
# Load the dataset
file_path_1 = 'enrol_01.csv'
df1 = pd.read_csv(file_path_1)

# Find unique states (initial check)
print("Unique states in Enrol 1 (Before Cleaning):")
print(df1['state'].unique())

# Clean null values explicitly for essential columns
df1.dropna(subset=['state', 'district'], inplace=True)

# Drop duplicates initially
df1.drop_duplicates(inplace=True)

# Clean State Names
df1['cleaned_state'] = df1['state'].replace(state_corrections)
df1.dropna(subset=['cleaned_state'], inplace=True)

# CLEAN DISTRICT Names -> Create 'new_district' column
df1['new_district'] = df1['district'].astype(str).str.strip()
df1['new_district'] = df1['new_district'].replace(district_corrections)

# Remove empty districts after cleaning
df1 = df1[df1['new_district'] != '']
df1 = df1[df1['new_district'] != 'nan']

# Clean duplicates AGAIN to remove redundant rows created by standardization
df1.drop_duplicates(inplace=True)

print("\nUnique states in Enrol 1 (After Cleaning):")
print(df1['cleaned_state'].unique())

## Step 4: Process Enrolment Data 2

In [ ]:
# Load the dataset
file_path_2 = 'enrol_02.csv'
df2 = pd.read_csv(file_path_2)

# Find unique states
print("Unique states in Enrol 2 (Before Cleaning):")
print(df2['state'].unique())

# Clean nulls explicitly
df2.dropna(subset=['state', 'district'], inplace=True)

# Drop duplicates initially
df2.drop_duplicates(inplace=True)

# Clean State Names
df2['cleaned_state'] = df2['state'].replace(state_corrections)
df2.dropna(subset=['cleaned_state'], inplace=True)

# CLEAN DISTRICT Names -> Create 'new_district' column
df2['new_district'] = df2['district'].astype(str).str.strip()
df2['new_district'] = df2['new_district'].replace(district_corrections)

# Remove empty districts
df2 = df2[df2['new_district'] != '']
df2 = df2[df2['new_district'] != 'nan']

# Clean duplicates AGAIN
df2.drop_duplicates(inplace=True)

print("\nUnique states in Enrol 2 (After Cleaning):")
print(df2['cleaned_state'].unique())

## Step 5: Process Enrolment Data 3

In [ ]:
# Load the dataset
file_path_3 = 'enrol_03.csv'
df3 = pd.read_csv(file_path_3)

# Find unique states
print("Unique states in Enrol 3 (Before Cleaning):")
print(df3['state'].unique())

# Clean nulls explicitly
df3.dropna(subset=['state', 'district'], inplace=True)

# Drop duplicates initially
df3.drop_duplicates(inplace=True)

# Clean State Names
df3['cleaned_state'] = df3['state'].replace(state_corrections)
df3.dropna(subset=['cleaned_state'], inplace=True)

# CLEAN DISTRICT Names -> Create 'new_district' column
df3['new_district'] = df3['district'].astype(str).str.strip()
df3['new_district'] = df3['new_district'].replace(district_corrections)

# Remove empty districts
df3 = df3[df3['new_district'] != '']
df3 = df3[df3['new_district'] != 'nan']

# Clean duplicates AGAIN
df3.drop_duplicates(inplace=True)

print("\nUnique states in Enrol 3 (After Cleaning):")
print(df3['cleaned_state'].unique())

## Step 6: Merge Datasets and Final Verification

In [ ]:
# Merge all 3 datasets
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

print(f"Merged Dataset Shape: {merged_df.shape}")

# Verify final unique states and districts
unique_states = merged_df['cleaned_state'].unique()
unique_districts = merged_df['new_district'].unique()
print(f"\nTotal Unique States: {len(unique_states)}")
print(sorted(unique_states))
print(f"\nTotal Unique Districts: {len(unique_districts)}")
# print(sorted(unique_districts)) # Corrected list


## Step 7: Display the Final Table

In [ ]:
# Show the first few rows of the merged dataset including the new columns
print("Top 5 rows of the merged dataset:")
print(merged_df[['date', 'cleaned_state', 'new_district', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']].head())


## Step 8: Define Canonical Chhattisgarh District List

In [ ]:
# Canonical list of 33 Chhattisgarh districts
chhattisgarh_canonical = [
    'Balod',
    'Baloda Bazar-Bhatapara',
    'Balrampur',
    'Bastar',
    'Bemetara',
    'Bijapur',
    'Bilaspur',
    'Dantewada',
    'Dhamtari',
    'Durg',
    'Gariaband',
    'Gaurela-Pendra-Marwahi',
    'Janjgir-Champa',
    'Jashpur',
    'Kabirdham (Kawardha)',
    'Kanker',
    'Khairagarh-Chhuikhadan-Gandai',
    'Kondagaon',
    'Korba',
    'Koriya (Korea)',
    'Mahasamund',
    'Manendragarh-Chirmiri-Bharatpur',
    'Mohla-Manpur-Ambagarh Chowki',
    'Narayanpur',
    'Raigarh',
    'Raipur',
    'Rajnandgaon',
    'Sakti',
    'Sarangarh-Bilaigarh',
    'Sukma',
    'Surajpur',
    'Surguja',
    'Korea'
]

print("Canonical Chhattisgarh District List defined.")

## Step 9: Chhattisgarh District Verification & Final Table

In [ ]:
# Filter for Chhattisgarh
cg_df = merged_df[merged_df['cleaned_state'] == 'Chhattisgarh']

# Verify against canonical list
cg_districts = cg_df['new_district'].unique()

print(f"Unique Districts in Chhattisgarh Data ({len(cg_districts)}):")
print(sorted(cg_districts))

print("\n--- Verification Against Canonical List ---")
missing = set(chhattisgarh_canonical) - set(cg_districts)
extra = set(cg_districts) - set(chhattisgarh_canonical)

if not missing and not extra:
    print("All Chhattisgarh districts map perfectly to your list!")
else:
    if missing: print(f"Missing from data: {sorted(missing)}")
    if extra: print(f"Extra in data (not in your list): {sorted(extra)}")

print("\n--- Final Table for Chhattisgarh Districts ---")
print(cg_df[['date', 'cleaned_state', 'new_district', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater']])


## Step 10: Export Chhattisgarh Data to Excel

In [ ]:
# Export to Excel
excel_filename = 'chhattisgarh_enrolment_data.xlsx'
cg_df.to_excel(excel_filename, index=False)
print(f"Successfully exported Chhattisgarh data to {excel_filename}")

## Step 11: Extract Unique Pincodes per District

In [ ]:
print("Unique Pincodes per District in Chhattisgarh:")
pincode_data = []

for district in sorted(chhattisgarh_canonical):
    # Using new_district to ensure we check against our standardized names
    # Note: Use cg_df which is already filtered for Chhattisgarh
    dist_df = cg_df[cg_df['new_district'] == district]
    
    if dist_df.empty:
        print(f"\n{district}: No data found.")
        pincode_data.append({'District': district, 'Unique Pincodes': 'No Data', 'Count': 0})
        continue

    # Clean pincodes
    # Ensure numeric validity (basic check)
    codes = dist_df['pincode'].dropna().unique()
    valid_codes = []
    for c in codes:
        try:
            c_str = str(c).split('.')[0] # handle float 123.0
            if c_str.isdigit() and len(c_str) >= 6:
                 valid_codes.append(int(c_str))
        except:
            continue
    
    valid_codes = sorted(list(set(valid_codes)))
    
    print(f"\n{district} ({len(valid_codes)} pincodes):")
    print(valid_codes)
    
    pincode_data.append({
        'District': district,
        'Unique Pincodes': ', '.join(map(str, valid_codes)),
        'Count': len(valid_codes)
    })

# Summary Table
pincode_df = pd.DataFrame(pincode_data)
print("\n--- Pincode Summary Table ---")
print(pincode_df)

## Step 12: Detailed Pincode Analysis (Multi-Use Checks)

In [ ]:
# Analysis 1: Districts using Multiple Pincodes (List the pincodes for each)
print("--- Analysis: Districts using Multiple Pincodes ---")
districts_with_multi_pincodes = pincode_df[pincode_df['Count'] > 1]
if districts_with_multi_pincodes.empty:
     print("No districts have more than 1 pincode.")
else:
     print(f"Found {len(districts_with_multi_pincodes)} districts with > 1 pincode.")

# Analysis 2: Pincodes used by Multiple Districts (Inconsistency Check)
print("\n--- Analysis: Pincodes used by Multiple Districts (Overlap Check) ---")
cg_df['pincode_clean'] = cg_df['pincode'].apply(lambda x: str(x).split('.')[0] if pd.notnull(x) else np.nan)
pincode_counts = cg_df.groupby('pincode_clean')['new_district'].nunique()
multi_district_pincodes = pincode_counts[pincode_counts > 1]

if multi_district_pincodes.empty:
    print("SUCCESS: No pincode is shared between different districts.")
else:
    print(f"WARNING: {len(multi_district_pincodes)} pincodes are mapped to multiple districts.")
    print("Exporting conflict details to Step 13...")

## Step 13: Export Pincode Conflicts to Excel

In [ ]:
# Find conflicting pincodes
conflicting_pincodes = multi_district_pincodes.index.tolist()

# Filter dataset for these pincodes
conflict_df = cg_df[cg_df['pincode_clean'].isin(conflicting_pincodes)]

if not conflict_df.empty:
    # Sort for easier reading
    conflict_df = conflict_df.sort_values(by=['pincode_clean', 'new_district'])
    
    # Export
    conflict_filename = 'chhattisgarh_pincode_conflicts.xlsx'
    conflict_df.to_excel(conflict_filename, index=False)
    print(f"Shared/Conflicting pincode details exported to {conflict_filename}")
    print("Top 10 rows of conflicts:")
    print(conflict_df[['new_district', 'pincode', 'date']].head(10))
else:
    print("No conflicting pincodes found to export.")

## Step 14: Reserved for Previous Summary Logic (Optional)

In [ ]:
# This step is kept as a placeholder if you want the specific single-row summary again, 
# but the next step (Step 15) provides the detailed breakdown you requested.
print("Proceeding to Detailed Analysis (Step 15)...")

## Step 15: Detailed Conflict Analysis (With State, Formatted Date, and Ages)

In [ ]:
print("Generating Detailed Dominance Analysis (One row per district for shared pincodes)...")

if not multi_district_pincodes.empty:
    detailed_data = []
    shared_pincodes = multi_district_pincodes.index.tolist()

    # Filter main DF for only shared pincodes
    all_conflicts = cg_df[cg_df['pincode_clean'].isin(shared_pincodes)]
    
    # Determine the 'Total Records' per pincode to help with sorting later
    pincode_volumes = all_conflicts['pincode_clean'].value_counts().to_dict()

    # Iterate through each shared pincode
    for pin in shared_pincodes:
        pin_df = all_conflicts[all_conflicts['pincode_clean'] == pin]
        total_pin_records = len(pin_df)
        
        # Get unique districts for this pincode
        districts = pin_df['new_district'].unique()
        
        # For each district sharing this pincode, calculate stats
        for dist in districts:
            dist_rows = pin_df[pin_df['new_district'] == dist]
            count = len(dist_rows)
            
            # Age Sums
            age_0_5 = dist_rows['age_0_5'].sum()
            age_5_17 = dist_rows['age_5_17'].sum()
            age_18_greater = dist_rows['age_18_greater'].sum()
            
            # Date Analysis (Find Mode and Range)
            dates = pd.to_datetime(dist_rows['date'], format='%d-%m-%Y', errors='coerce')
            valid_dates = dates.dropna()
            
            if not valid_dates.empty:
                date_mode = valid_dates.mode()[0].strftime('%d-%m-%Y')
                month_year = valid_dates.mode()[0].strftime('%B %Y') # e.g. January 2024
            else:
                date_mode = "Unknown"
                month_year = "Unknown"
                
            detailed_data.append({
                'State': 'Chhattisgarh', # Restored State Column
                'Pincode': pin,
                'Total Pincode Volume': total_pin_records, # Helper for sorting
                'District': dist,
                'Record Count': count,
                'Age_0_5_Sum': age_0_5,
                'Age_5_17_Sum': age_5_17,
                'Age_18_Greater_Sum': age_18_greater,
                'Representative_Date': date_mode,
                'Month_Year': month_year
            })
            
    # Convert to DataFrame
    detailed_df = pd.DataFrame(detailed_data)
    
    # Sorting Logic: 
    # 1. Primary Sort: Pincode Volume (High to Low) -> Shows biggest conflict groups first
    # 2. Secondary Sort: Record Count (High to Low) -> Shows Dominant District first within group
    detailed_df = detailed_df.sort_values(by=['Total Pincode Volume', 'Record Count'], ascending=[False, False])
    
    print("Detailed Dominance Table Generated (Top 10 rows):")
    print(detailed_df.head(10))
    
    # Export
    output_file = 'chhattisgarh_detailed_dominance.xlsx'
    detailed_df.to_excel(output_file, index=False)
    print(f"\nDetailed Excel file created at: {output_file}")

else:
    print("No shared pincodes found.")

## Step 16: District-wise Enrolment Ranking (Aggregated by Total Ages)

In [ ]:
print("Generating District-wise Enrolment Ranking Table...")

# Group by District
district_stats = cg_df.groupby('new_district').agg({
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum'
}).reset_index()

# Calculate Total Enrolments (Sum of all 3 age categories)
district_stats['Total_Enrolment'] = district_stats['age_0_5'] + district_stats['age_5_17'] + district_stats['age_18_greater']

# Add Representative Date info for each district (Optional but consistent with prev requests)
# Using the Mode date for each district
date_modes = cg_df.groupby('new_district')['date'].apply(lambda x: pd.to_datetime(x, format='%d-%m-%Y', errors='coerce').mode()[0].strftime('%d-%m-%Y') 
                                                       if not pd.to_datetime(x, format='%d-%m-%Y', errors='coerce').mode().empty else 'Unknown')
district_stats['Most_Frequent_Date'] = district_stats['new_district'].map(date_modes)

# Add State column
district_stats.insert(0, 'State', 'Chhattisgarh')

# Rename columns for clarity
district_stats = district_stats.rename(columns={
    'new_district': 'District',
    'age_0_5': 'Enrolment_Age_0_5',
    'age_5_17': 'Enrolment_Age_5_17',
    'age_18_greater': 'Enrolment_Age_18_Plus'
})

# Sort by Total Enrolment Descending (Highest First)
district_stats = district_stats.sort_values(by='Total_Enrolment', ascending=False)

print("Top 10 Districts by Total Enrolment:")
print(district_stats.head(10))

# Export to Excel
ranking_filename = 'chhattisgarh_district_ranking.xlsx'
district_stats.to_excel(ranking_filename, index=False)
print(f"\nDistrict Ranking Excel created at: {ranking_filename}")